# Scrape Wallstreet online

In [1]:
from ModulScrapeWallstreet import scrape_futureprice, future_price_to_sql


In [2]:
future_df = scrape_futureprice()

C:\Users\Lenovo\Documents\GitHub\ContractCalculator\ModulScrapeWallstreet.py:15: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 15 of the file C:\Users\Lenovo\Documents\GitHub\ContractCalculator\ModulScrapeWallstreet.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(page.content, 'html')
C:\Users\Lenovo\Documents\GitHub\ContractCalculator\ModulScrapeWallstreet.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  future_price_today = future_price_today.append(ser, ignore_index=True)
C:\Users\Lenovo\Documents\GitHub\Contract

In [3]:
future_df

,kontrakt,price,absolut_inc,perc_inc,date_fullfillment,date_price,currency,commodity_id
0,Mai 2022,351.80,"3,15","+0,90 %",2022-05-23,2022-05-15,GBP,2
1,Jul 2022,347.60,"3,15","+0,91 %",2022-07-07,2022-05-15,GBP,2
2,Nov 2022,336.30,"3,15","+0,95 %",2022-11-23,2022-05-15,GBP,2
3,Jan 2023,338.70,"3,55","+1,06 %",2023-01-23,2022-05-15,GBP,2
4,Mär 2023,340.40,"3,45","+1,02 %",2023-03-23,2022-05-15,GBP,2
5,Mai 2023,342.30,"3,35","+0,99 %",2023-05-23,2022-05-15,GBP,2
6,Jul 2023,316.85,"4,35","+1,39 %",2023-07-07,2022-05-15,GBP,2
7,Nov 2023,286.80,"5,35","+1,90 %",2023-11-23,2022-05-15,GBP,2
8,Jan 2024,287.75,"5,35","+1,89 %",2024-01-23,2022-05-15,GBP,2
9,Mär 2024,289.25,"5,35","+1,88 %",2024-03-22,2022-05-15,GBP,2


In [4]:
future_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   kontrakt           10 non-null     object        
 1   price              10 non-null     float64       
 2   absolut_inc        10 non-null     object        
 3   perc_inc           10 non-null     object        
 4   date_fullfillment  10 non-null     datetime64[ns]
 5   date_price         10 non-null     datetime64[ns]
 6   currency           10 non-null     object        
 7   commodity_id       10 non-null     int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 768.0+ bytes


In [5]:
future_price_to_sql(future_df)

InterfaceError: Error binding parameter 1 - probably unsupported type.

In [6]:
future_df.to_csv('Data/future_df')